<a href="https://colab.research.google.com/github/Dan5Playground/colab/blob/master/fast_text_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sentiment analysis with fasttext
- corpus : [sentiment140](http://help.sentiment140.com/for-students/) dataset that contains 1,600,000 tweets extracted using the twitter api. The extracted tweets have been annotated with 2 keys: 0 = negative and 4 = positive. 
- method : [FastText](https://github.com/facebookresearch/fastText) , Facebook’s open source library for fast text representation and classification.

# Installing FastText
- FastText Version: 0.2.0
- This will generate object files for all the classes and the main binary fasttext

In [0]:
!wget https://github.com/facebookresearch/fastText/archive/v0.2.0.zip
!unzip v0.2.0.zip
%cd fastText-0.2.0
!make

In [4]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 3531 (delta 15), reused 31 (delta 10), pack-reused 3483
Receiving objects: 100% (3531/3531), 8.04 MiB | 30.06 MiB/s, done.
Resolving deltas: 100% (2208/2208), done.
/content/fastText
Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2824913 sha256=ec391de0fadb8b16f075f61368e220e0d10e282286b06ce3e179055783b87f7a
  Stored in directory: /tmp/pip-ephem-wheel-cache-rj4ucsgp/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [5]:
!pip install kaggle
!mkdir /root/.kaggle

!ls /root/.kaggle

open('/root/.kaggle/kaggle.json','w').write('{"username":"vaqasam","key":"2ec5d91c66a678bc314920e84c687f3b"}')
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
%cd /content
!kaggle datasets download kazanova/sentiment140


/content
 93% 75.0M/80.9M [00:02<00:00, 34.0MB/s]
100% 80.9M/80.9M [00:02<00:00, 33.7MB/s]


In [5]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [0]:
import csv  
import re

train = open('tweets.train','w')  
test = open('tweets.valid','w')  
with open('training.1600000.processed.noemoticon.csv', mode='r', encoding = "ISO-8859-1") as csv_file:  
    csv_reader = csv.DictReader(csv_file, fieldnames=['target', 'id', 'date', 'flag', 'user', 'text'])
    line = 0
    for row in csv_reader:
        # Clean the training data
        # First we lower case the text
        text = row["text"].lower()
        # remove links
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
        #Remove usernames
        text = re.sub('@[^\s]+','', text)
        # replace hashtags by just words
        text = re.sub(r'#([^\s]+)', r'\1', text)
        #correct all multiple white spaces to a single white space
        text = re.sub('[\s]+', ' ', text)
        # Additional clean up : removing words less than 3 chars, and remove space at the beginning and teh end
        text = re.sub(r'\W*\b\w{1,3}\b', '', text)
        text = text.strip()
        line = line + 1
        # Split data into train and validation
        if line%16 == 0:
            print(f'__label__{row["target"]} {text}', file=test)
        else:
            print(f'__label__{row["target"]} {text}', file=train)

In [43]:
!./fastText-0.2.0/fasttext supervised -input tweets.train -output model_tweet -lr 0.1 -epoch 30

Read 13M words
Number of words:  578539
Number of labels: 2
Progress: 100.0% words/sec/thread:  279158 lr:  0.000000 loss:  0.314597 ETA:   0h 0m


In [44]:
# test the model 
! ./fastText-0.2.0/fasttext test model_tweet.bin tweets.valid

N	99880
P@1	0.753
R@1	0.753


In [0]:
# some basic analysis for the training data 
import pandas as pd 
data =  pd.read_csv('training.1600000.processed.noemoticon.csv', header = None, encoding = "ISO-8859-1", error_bad_lines=False) #pd.read_csv("tweets.valid")
data[0].unique()

array([0, 4])

In [28]:
def process_text(row):
  if row['target'][0]
data.apply (lambda row: process_text(row), axis=1)

0    __label__0 wish watch with miss premiere?!
Name: 0, dtype: object

In [25]:
from fasttext import load_model

classifier = load_model("/content/model_tweet.bin")  
texts = "The only reason people make Coming Out videos is to get attention."

labels = classifier.predict(texts)  
print (labels)  

texts = "I am gay"
labels = classifier.predict(texts)  
print (labels[0][0])  
texts = "this video make my heart melted "
labels = classifier.predict(texts)  
print (labels)  
 

texts = "And dont come at me saying that's wrong because it is against the bible"
labels = classifier.predict(texts)  
print (labels)  

# texts = ['Ugghhh... Not happy at all! sorry', 'Happyyyyyyy', 'OH yeah! lets rock.']
# And dont come at me saying that's wrong because it is against the bible

(('__label__4',), array([0.53116578]))
__label__4
(('__label__0',), array([0.63748825]))
(('__label__0',), array([0.78817308]))


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# save the model 
!cp /content/model_tweet.bin "/content/gdrive/My Drive/model_tweet.bin"

## dataset 2 : 

IMDB 

In [0]:
# data set 2 
train the model with IMDB moive reviews



In [7]:
!ls

sample_data  sentiment140.zip


In [8]:
!kaggle datasets download iarunava/imdb-movie-reviews-dataset

 91% 203M/224M [00:05<00:00, 26.0MB/s]
100% 224M/224M [00:05<00:00, 44.0MB/s]


In [0]:
!unzip imdb-movie-reviews-dataset.zip

In [14]:
!ls aclimdb/aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [13]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [51]:
import csv  
import re
import os

#train = open('imdb.train','w')  
test = open('imdb.valid','w')  
# open all training files 
path = 'aclimdb/aclImdb/test/'
for folder in ['neg', 'pos']: # os.listdir(path):
  #print(folder)
  i = 0
  for filename in os.listdir(path + '/' +folder + '/'):
    i+=1
    if i < 5:
      print (i)
    
    with open(path + '/' +folder + '/' + filename) as file: # Use file to refer to the file object
      data = file.read()
      # print (data)
      # Clean the training data
      # First we lower case the text
      text = data.lower()
      # remove links
      text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
      #Remove usernames
      text = re.sub('@[^\s]+','', text)
      # replace hashtags by just words
      text = re.sub(r'#([^\s]+)', r'\1', text)
      #correct all multiple white spaces to a single white space
      text = re.sub('[\s]+', ' ', text)
      # Additional clean up : removing words less than 3 chars, and remove space at the beginning and teh end
      text = re.sub(r'\W*\b\w{1,3}\b', '', text)
      text = text.strip()
      target = 0 if folder == 'neg' else 1
      print(f'__label__{target} {text}', file=test)
  print (folder, i)
#train.close()
test.close()


1
2
3
4
neg 12500
1
2
3
4
pos 12500


In [2]:
%cd /content
!ls

/content
fastText-0.2.0	sample_data  v0.2.0.zip


In [58]:
!./fastText-0.2.0/fasttext supervised -input imdb.train -output model_imdb -lr 0.1 -epoch 90

Read 3M words
Number of words:  184290
Number of labels: 2
Progress: 100.0% words/sec/thread:  347150 lr:  0.000000 loss:  0.029013 ETA:   0h 0m


In [60]:
# test the model 
! ./fastText-0.2.0/fasttext test model_imdb.bin imdb.valid

N	25000
P@1	0.856
R@1	0.856


In [0]:
# save the model 
!cp /content/model_imdb.bin "/content/gdrive/My Drive/model_imdb'[''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
].bin"

In [12]:
from fasttext import load_model

classifier = load_model("/content/gdrive/My Drive/model_tweet.bin") 
classifier2 = load_model("/content/gdrive/My Drive/model_imdb.bin") 
texts = ["the only reason people make Coming Out videos is to get attention.", "i am gay", "i am lesbian", "i hate you", "it is really bad "]

labels = classifier.predict(texts)  
print (labels)
labels = classifier2.predict(texts) 
print (labels)  

([['__label__4'], ['__label__4'], ['__label__4'], ['__label__0'], ['__label__0']], [array([0.5617779], dtype=float32), array([0.8204645], dtype=float32), array([0.6290098], dtype=float32), array([0.9947708], dtype=float32), array([0.59370524], dtype=float32)])
([['__label__0'], ['__label__0'], ['__label__0'], ['__label__1'], ['__label__0']], [array([0.99919814], dtype=float32), array([1.00001], dtype=float32), array([1.00001], dtype=float32), array([1.00001], dtype=float32), array([0.99996376], dtype=float32)])


In [15]:
from google.colab import files
uploaded = files.upload()

Saving comments-Eh7WRYXVh9M.csv to comments-Eh7WRYXVh9M.csv


In [0]:
import pandas as pd
data = pd.read_csv("comments-Eh7WRYXVh9M.csv")

In [24]:
grouped = data.groupby(['id'])
i = 0
for name, group in grouped:
  if i < 100 and (not group.iloc[0]['replies.commentText']):
    print (group.iloc[0]['replies.commentText'])
  i += 1
print (i)

100349


In [26]:
data.columns

Index(['id', 'user', 'date', 'timestamp', 'commentText', 'likes', 'hasReplies',
       'numberOfReplies', 'replies.id', 'replies.user', 'replies.date',
       'replies.timestamp', 'replies.commentText', 'replies.likes'],
      dtype='object')

In [0]:
import re
def preporcess_text(text):
  # remove links
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
  #Remove usernames
  text = re.sub('@[^\s]+','', text)
  # replace hashtags by just words
  text = re.sub(r'#([^\s]+)', r'\1', text)
  #correct all multiple white spaces to a single white space
  text = re.sub('[\s]+', ' ', text)
  # Additional clean up : removing words less than 3 chars, and remove space at the beginning and teh end
  text = re.sub(r'\W*\b\w{1,3}\b', '', text)
  text = text.strip()
  return text


In [51]:
data['sentiment_imdb'] = pd.Series(-1, index=data.index)
data['sentiment_twitter'] = pd.Series(-1, index=data.index)
data['sentiment_imdb_reply'] = pd.Series(-1, index=data.index)
data['sentiment_twitter_reply'] = pd.Series(-1, index=data.index)
print (data.columns)

Index(['id', 'user', 'date', 'timestamp', 'commentText', 'likes', 'hasReplies',
       'numberOfReplies', 'replies.id', 'replies.user', 'replies.date',
       'replies.timestamp', 'replies.commentText', 'replies.likes',
       'sentiment1', 'sentiment_imdb', 'sentiment_twitter',
       'sentiment_imdb_reply', 'sentiment_twitter_reply'],
      dtype='object')


In [67]:
for idx, row in data.iterrows():
  #if idx > 100:
  #  break
  # First we lower case the text
  text = str(row['commentText'])
  text = preporcess_text(text)
  #print(row['replies.commentText'], len(str(row['replies.commentText'])))
  if not str(row['replies.commentText']) == 'nan':
    reply = row['replies.commentText']
    #print("replies", reply)
    reply = preporcess_text(reply)
    label3 = classifier.predict(reply) 
    v = 0 if label3[0][0] == '__label__0' else 1
    data.at[idx,'sentiment_twitter_reply'] = v
    label4 = classifier2.predict(reply)
    v = 0 if label4[0][0] == '__label__0' else 1
    data.at[idx,'sentiment_imdb_reply'] = v
  if len(text) > 0:
    label1 = classifier.predict(text)
    v = 0 if label1[0][0] == '__label__0' else 1
    data.at[idx,'sentiment_twitter'] = v
    #row['sentiment_twitter'] = 0 if label1[0][0] == '__label__0' else 1
    label2 = classifier2.predict(text) 
    v = 0 if label2[0][0] == '__label__0' else 1
    data.at[idx,'sentiment_imdb'] = v
    #row['sentiment_imdb'] =  0 if label2[0][0] == '__label__0' else 1

    
    if idx % 10000 == 0:
      print (text)
      print (label1[0], label2[0])


  


crying shiit
('__label__0',) ('__label__0',)
GIRL!!! 👌👌👌
('__label__4',) ('__label__0',)
that care that much boyfriend recommended videos. Whats with that dawgs..
('__label__4',) ('__label__0',)
really disagree with this your person proud that this have guts!
('__label__4',) ('__label__1',)
GIRL!!!! XOXOXO
('__label__4',) ('__label__0',)
GOOD BEAUTIFUL PROUD! YOUR GORGEOUS PERSON DESERVE HAPPY
('__label__4',) ('__label__0',)
sweet little cinnamon sweet pure this world didnt bring tissues!"
('__label__4',) ('__label__1',)
this unexpected insanely proud Ingrid amazing happy finally yourself makes happy that these youtubers starting come truly amazing proud every single them
('__label__4',) ('__label__1',)


In [85]:
data.sentiment_imdb_reply.value_counts()

-1    100352
 0     28195
 1     16296
Name: sentiment_imdb_reply, dtype: int64

In [89]:
28195/ 44491#(64485+ 34010)

0.6337236744510125

In [86]:
data.sentiment_twitter_reply.value_counts()

-1    100352
 1     30024
 0     14467
Name: sentiment_twitter_reply, dtype: int64

In [70]:
# correltion 
data.corr()

,timestamp,likes,numberOfReplies,replies.timestamp,replies.likes,sentiment1,sentiment_imdb,sentiment_twitter,sentiment_imdb_reply,sentiment_twitter_reply
timestamp,1.000000,0.014180,0.032766,NaN,NaN,NaN,-0.137495,-0.119627,NaN,NaN
likes,0.014180,1.000000,0.415804,NaN,NaN,NaN,-0.005446,-0.008194,NaN,NaN
numberOfReplies,0.032766,0.415804,1.000000,NaN,NaN,NaN,-0.040789,-0.047242,NaN,NaN
replies.timestamp,NaN,NaN,NaN,1.000000,-0.052240,NaN,NaN,NaN,-0.039057,0.007130
replies.likes,NaN,NaN,NaN,-0.052240,1.000000,NaN,NaN,NaN,0.021756,-0.023219
sentiment1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sentiment_imdb,-0.137495,-0.005446,-0.040789,NaN,NaN,NaN,1.000000,0.867005,-0.796848,-0.820333
sentiment_twitter,-0.119627,-0.008194,-0.047242,NaN,NaN,NaN,0.867005,1.000000,-0.841952,-0.866765
sentiment_imdb_reply,NaN,NaN,NaN,-0.039057,0.021756,NaN,-0.796848,-0.841952,1.000000,0.878983
sentiment_twitter_reply,NaN,NaN,NaN,0.007130,-0.023219,NaN,-0.820333,-0.866765,0.878983,1.000000


In [90]:
from scipy.stats import ttest_ind

cat1 = data[data['sentiment_imdb']==0]
cat2 = data[data['sentiment_imdb']==1]
cat11 = data[data['sentiment_twitter']==0]
cat22 = data[data['sentiment_twitter']==1]

print ('neg: ', cat1['likes'].mean(), cat1['likes'].std())
print ('pos: ', cat2['likes'].mean(), cat2['likes'].std())
print (ttest_ind(cat1['likes'], cat2['likes']))
#print (ttest_ind(cat11['likes'], cat22['likes']))
print ()
print("twitter: ")
print ('neg: ', cat11['likes'].mean(), cat11['likes'].std())
print ('pos: ', cat22['likes'].mean(), cat22['likes'].std())
#print (ttest_ind(cat1['likes'], cat2['likes']))
print (ttest_ind(cat11['likes'], cat22['likes']))

neg:  5.704969126727433 112.30425865919187
pos:  4.092191982631619 93.02587603143334
Ttest_indResult(statistic=2.404098807363241, pvalue=0.01621422548763548)

twitter: 
neg:  7.414702233250621 72.42775406741615
pos:  4.107872534142641 104.66085895473967
Ttest_indResult(statistic=3.8358095933009033, pvalue=0.0001252292741539345)


In [77]:
print ('neg: ', cat1['numberOfReplies'].mean(), cat1['numberOfReplies'].std())
print ('pos: ', cat2['numberOfReplies'].mean(), cat2['numberOfReplies'].std())
print (ttest_ind(cat1['numberOfReplies'], cat2['numberOfReplies']))
print (ttest_ind(cat11['numberOfReplies'], cat22['numberOfReplies']))
print ()
print("twitter: ")
print ('neg: ', cat11['numberOfReplies'].mean(), cat11['numberOfReplies'].std())
print ('pos: ', cat22['numberOfReplies'].mean(), cat22['numberOfReplies'].std())
print (ttest_ind(cat1['numberOfReplies'], cat2['numberOfReplies']))
print (ttest_ind(cat11['numberOfReplies'], cat22['numberOfReplies']))

neg:  0.7417524257571303 5.342526193282308
pos:  0.2951694192447856 3.5125316105595714
Ttest_indResult(statistic=15.736137761600789, pvalue=1.0000495482865161e-55)
Ttest_indResult(statistic=20.142295669750897, pvalue=4.779180073911673e-90)

twitter: 
neg:  1.0632754342431763 6.072424416981323
pos:  0.3292382397572079 3.767304309126861
Ttest_indResult(statistic=15.736137761600789, pvalue=1.0000495482865161e-55)
Ttest_indResult(statistic=20.142295669750897, pvalue=4.779180073911673e-90)


In [93]:
cat3 = data[data['sentiment_imdb_reply']==0]
cat4 = data[data['sentiment_imdb_reply']==1]
cat33 = data[data['sentiment_twitter_reply']==0]
cat44 = data[data['sentiment_twitter_reply']==1]
print ('neg: ', cat3['replies.likes'].mean(), cat3['replies.likes'].std())
print ('pos: ', cat4['replies.likes'].mean(), cat4['replies.likes'].std())
print (ttest_ind(cat3['replies.likes'], cat4['replies.likes']))
#print (ttest_ind(cat11['likes'], cat22['likes']))
print ()
print("twitter: ")
print ('neg: ', cat33['replies.likes'].mean(), cat33['replies.likes'].std())
print ('pos: ', cat44['replies.likes'].mean(), cat44['replies.likes'].std())
#print (ttest_ind(cat1['likes'], cat2['likes']))
print (ttest_ind(cat33['replies.likes'], cat44['replies.likes']))

neg:  1.9443163681503812 6.473174291777249
pos:  2.2306701030927836 6.148803814161286
Ttest_indResult(statistic=-4.578146965341844, pvalue=4.703810383768535e-06)

twitter: 
neg:  2.262666758830442 6.598779189429942
pos:  1.9463429256594724 6.23573630265293
Ttest_indResult(statistic=4.917353894214381, pvalue=8.803401445904629e-07)


In [0]:
data.to_csv('comments_with_sentiment.csv')

In [0]:
# download the comments
files.download('comments_with_sentiment.csv') 

In [98]:
texts = ['i am gay', 'i am lesbian', 'im crying too shiit', "i am bisexual","And dont come at me saying that's wrong because it is against the bible", "i am gay and proud it", "u made my heart melted", "stay strong", "I found it hard to understand if I was bi or lesbian but I found out eventually so ya lesbian squad", "i just wanna write that i'm lesbian.", "The only reason people make Coming Out videos is to get attention.", "Go to hell Wtf", "this was made on my birthday yay !!!!"]
labels1 = classifier.predict(texts)
labels2 = classifier2.predict(texts)

for i, t in enumerate(texts):
  print (t, labels1[0][i],labels1[1][i], labels2[0][i], labels2[1][i])


i am gay ['__label__4'] [0.8204645] ['__label__0'] [1.00001]
i am lesbian ['__label__4'] [0.6290098] ['__label__0'] [1.00001]
im crying too shiit ['__label__0'] [0.65832204] ['__label__0'] [1.0000099]
i am bisexual ['__label__0'] [0.8742035] ['__label__0'] [1.00001]
And dont come at me saying that's wrong because it is against the bible ['__label__0'] [0.81614846] ['__label__0'] [1.00001]
i am gay and proud it ['__label__4'] [0.99981827] ['__label__1'] [0.9999856]
u made my heart melted ['__label__0'] [0.7135764] ['__label__1'] [1.00001]
stay strong ['__label__4'] [0.57202417] ['__label__0'] [0.9999591]
I found it hard to understand if I was bi or lesbian but I found out eventually so ya lesbian squad ['__label__0'] [0.79947996] ['__label__0'] [1.00001]
i just wanna write that i'm lesbian. ['__label__0'] [0.8546706] ['__label__0'] [1.00001]
The only reason people make Coming Out videos is to get attention. ['__label__4'] [0.5617779] ['__label__0'] [0.99919814]
Go to hell Wtf ['__label_